<a href="https://colab.research.google.com/github/colomba-o/P6-image-classification/blob/main/P5_modele_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# PROJET 5 : Catégorisez automatiquement des questions

## Modèle final sélectionné

Enregistrement du modèle afin de le tester avec une REST API.

In [ ]:
import nltk
import numpy as np
import pandas as pd
import pickle
import warnings
import re
import scipy
import string

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
from sklearn import model_selection
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
l = ["use", "using", "used", "get", "got", "getting", 
     "taking", "like", "would", "want", "wait", "need", "needs", "try", "tried", "trying", "if", "a", "b", 
     "d", "e", "f", "g", "h" "i", "j", "k", "l", "m", "n", "o", "p", "q", "s", "t", "u", "v", "w", "x", 
     "y", "z", "http", "www", "follow", "write", "way","also","make", "one","example","number","good", "best" , 
     "better", "without", "com", " ", "--", "work", "see", "look", "know", "seem", "find", "create", "give",
     "could", "think", "div", "understand", "+", "something", "line", "set", "question", "answer", "thing", 
     "mean", "solution"]

stop_words= stop_words + l
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
def remove_braquet_tags():
    df["tags"] = df["tags"].apply(lambda text: preprocessing_tag(text))


def count_tags(data):
    """ Fonction qui permet de compter les différents tags, de les classer et de donner le pourcentage
    """
    count_df = pd.Series(data.loc[:, question_tags_featured].squeeze().values.reshape(-1)).value_counts()
    ct_df = pd.DataFrame({'Tag': count_df.index,
                          'Count': count_df.values,
                          'Pourcentage (%)': (100 * (count_df / count_df.sum())).values})
    return ct_df


def cloud(text, color_back, title):
    wordcloud = WordCloud(background_color=color_back, stopwords=stop_words, max_words=100).generate(" ".join([i for i in text.str.upper()]))
    #wordcloud = WordCloud(background_color=color_back, max_words=100).generate(" ".join([i for i in text.str.upper()]))
    plt.figure(figsize=(15,10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(title)
    

def replace_words(string_to_process):
    string_to_process = re.sub("c#","csharp",string_to_process)
    string_to_process = re.sub(".net","dotnet",string_to_process)
    string_to_process = re.sub(".js","js",string_to_process)
    string_to_process = re.sub("dos2unix","dostounix",string_to_process)
    string_to_process = re.sub(".io","io",string_to_process)
    string_to_process = re.sub("sqlserver2008r2","sqlserverrtwo",string_to_process)
    string_to_process = re.sub("request.form","requestform",string_to_process)
    string_to_process = re.sub("html5","htmlfive",string_to_process)
    string_to_process = re.sub("system.diagnostics","systemdiagnostics",string_to_process)
    string_to_process = re.sub(".conf","conf",string_to_process)
    string_to_process = re.sub(".py","py",string_to_process)
    string_to_process = re.sub(".out","out",string_to_process)
    return string_to_process


def preprocessing_tag(tags_string):
    tags_string = tags_string.lower()
    tags_string = tags_string.replace("><", " ")
    tags_string = tags_string.replace(">", "")
    tags_string = tags_string.replace("<", "")
    tags_string = tags_string.replace("-", "")
    tags_string = replace_words(tags_string)
    tags_string = re.sub("[^a-zA-Z]", " ", tags_string) 
    tags = tags_string.split()
    tags = set(tags)
    return ' '.join(str(lemma.lemmatize(j)) for j in tags if (len(j)!=1 or j=='c' or j=='r'))


def preprocessing_body(string_to_process):
    string_to_process = BeautifulSoup(string_to_process, "lxml").text
    return preprocessing_text(string_to_process)

def remove_html(string_to_process):
    string_to_process = BeautifulSoup(string_to_process, "lxml").text
    return string_to_process

def preprocessing_text(string_to_process):
    #print("avant "+string_to_process)
    # all the string in lower 
    string_to_process = string_to_process.lower() 
    #print("lower "+string_to_process)
    string_to_process = string_to_process.replace("-", "")
    string_to_process = replace_words(string_to_process)
    
    # remove ponctuation
    tokenizer = RegexpTokenizer(r'\w+')
    #words = tokenizer.tokenize(string_to_process)
    
    # 2. Retirer ce qui n'est pas une lettre alphabétique       
    letters_numbers_only = re.sub("[^a-zA-Z#+\-]", " ", string_to_process) 

    # 3. Découpage
    words = letters_numbers_only.split()    

    # 4. remove stop words and keep only nouns
    words =' '.join(str(lemma.lemmatize(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c' or j=='r'))
    return words.lstrip()


def lemmatization(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:  
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))


def remove_stopwords(text): 
    words=token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    return ' '.join(map(str, filtered))


def keep_nouns_only(text): 
    words=token.tokenize(text)
    filtered = [w for w in words if (nltk.pos_tag([w])[0][1][0] == "N")]
    return ' '.join(map(str, filtered))


def length_nb_question(data_df):
    len_post_before = 0
    questions_before = 0
    for question in data_df:   
        len_post_before +=len(question)
        questions_before += 1
    return questions_before, len_post_before


def print_summerize_questions(questions_before, questions_after, len_post_before, len_post_after):
    print("number of questions before=",questions_before) 
    print("number of questions completed=",questions_after)        
    no_dup_avg_len_post_before=(len_post_before*1.0)/questions_before
    no_dup_avg_len_post=(len_post_after*1.0)/questions_after
    print( "Avg. length of questions(Title+Body) before processing: %d"%no_dup_avg_len_post_before)
    print( "Avg. length of questions(Title+Body) after processing: %d"%no_dup_avg_len_post)


def target_processing(tags):
    new_tags = ""
    tags_words = tags.split(" ")
    for tag in tags_words : 
      
      if (tag in SELECTED_TAGS) :
        new_tags = new_tags + ' ' + tag 
      new_tags.lstrip()  
    return new_tags

    
def nb_apparition_tag_questions(nb_tags):
    tags_most_common = tags_frequence.most_common(nb_tags)
    tags_df = pd.DataFrame(tags_most_common, columns = ['tags' , 'nb_tags']) 
    tags_df.columns
    tags_df = tags_df.sort_values(['nb_tags'], ascending=False)
    return tags_df


def list_n_first_tags(n):
  new_df = tags_df.iloc[:n]
  new_tags_df = new_df[["tags"]].transpose()
  list_first_tags = new_tags_df.to_numpy()
  list_first_tags = list_first_tags[0]
  list_first_tags = list_first_tags.tolist()
  return list_first_tags

## Data

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/data_p5.csv',index_col=None, header=0, encoding='utf-8')
lemma = nltk.wordnet.WordNetLemmatizer()
token = ToktokTokenizer()

In [ ]:
df.head()

,title,body,tags
0,How can I remove duplicate rows?,<p>What is the best way to remove duplicate ro...,<sql-server><tsql><duplicates>
1,How can I pass arguments to a batch file?,<p>I need to pass an ID and a password to a ba...,<batch-file><arguments>
2,Where can I learn jQuery? Is it worth it?,<p>I've had a lot of good experiences learning...,<javascript><jquery><review>
3,Getting all types that implement an interface,"<p>Using reflection, how can I get all types t...",<c#><optimization><reflection><lambda><c#-3.0>
4,Why am I getting a NoClassDefFoundError in Java?,<p>I am getting a <code>NoClassDefFoundError</...,<java><noclassdeffounderror>


## Paramètres du notebook

In [ ]:
nb_selected_tags = 200
MIN_DF = 1

## Pre-processing

### Tags

In [ ]:
remove_braquet_tags()

In [ ]:
df["nb_tags"] = df["tags"].apply(lambda text: len(text.split()))

In [ ]:
df['tags_list'] = df['tags'].apply(lambda x: x.split())
tags_list = [item for sublist in df['tags_list'].values for item in sublist]

In [ ]:
tags_unique_list = list(set(tags_list))

In [ ]:
tags_frequence= nltk.FreqDist(tags_list)
words_most_common = tags_frequence.most_common(20)

In [ ]:
tags_df = nb_apparition_tag_questions(len(tags_unique_list))

In [ ]:
SELECTED_TAGS = list_n_first_tags(nb_selected_tags)

In [ ]:
df['new_tags'] = df["tags"].apply(lambda text : target_processing(text))

In [ ]:
selected_tags_df = pd.DataFrame(SELECTED_TAGS, columns=['selected_tags'])

In [ ]:
df = df[df["new_tags"] != ""]

In [ ]:
df["nb_new_tags"] = df["new_tags"].apply(lambda text: len(text.split()))

In [ ]:
df.head()

,title,body,tags,nb_tags,tags_list,new_tags,nb_new_tags
0,How can I remove duplicate rows?,<p>What is the best way to remove duplicate ro...,sqlserver duplicate tsql,3,"[sqlserver, duplicate, tsql]",sqlserver tsql,2
1,How can I pass arguments to a batch file?,<p>I need to pass an ID and a password to a ba...,batchfile argument,2,"[batchfile, argument]",batchfile,1
2,Where can I learn jQuery? Is it worth it?,<p>I've had a lot of good experiences learning...,review javascript jquery,3,"[review, javascript, jquery]",javascript jquery,2
3,Getting all types that implement an interface,"<p>Using reflection, how can I get all types t...",csharp reflecion optimizaion lambda,4,"[csharp, reflecion, optimizaion, lambda]",csharp reflecion optimizaion lambda,4
4,Why am I getting a NoClassDefFoundError in Java?,<p>I am getting a <code>NoClassDefFoundError</...,noclassdeffounderror java,2,"[noclassdeffounderror, java]",java,1


### Corpus

In [ ]:
df['body'] = df['body'].apply(lambda x: remove_html(x))

In [ ]:
df['title_title_title_body'] = df['title'] + " " + df['title'] + " " + df['title'] + " " + df['body'] 

In [ ]:
df['title_title_title_body'] = df['title_title_title_body'].apply(lambda x: preprocessing_text(x))

In [ ]:
#df['title_title_title_body'] = df['title_title_title_body'].apply(lambda x: keep_nouns_only(x))

In [ ]:
df['title_title_title_body'] = df['title_title_title_body'].apply(lambda x: lemmatization(x))

In [ ]:
print(df.shape)
df = df[df['body'] != ""]
print(df.shape)

(7747, 8)
(7747, 8)


### Modele

In [ ]:
X_tttb = df['title_title_title_body']
y = df['new_tags']

In [ ]:
print(SELECTED_TAGS)

['javascript', 'python', 'java', 'git', 'c', 'android', 'csharp', 'html', 'cs', 'jquery', 'io', 'dotnet', 'php', 'bash', 'string', 'nodejs', 'sql', 'mysql', 'linux', 'array', 'objectivec', 'algorithm', 'shell', 'performance', 'sqlserver', 'oop', 'angulajs', 'database', 'window', 'languageagnostic', 'http', 'github', 'multithreading', 'list', 'r', 'verioncontrol', 'macos', 'iphone', 'vim', 'panda', 'rest', 'xcode', 'unix', 'ruby', 'commandline', 'dataframe', 'faq', 'cocoatouch', 'security', 'designpatterns', 'rubyonrails', 'swift', 'regex', 'docker', 'visualstuio', 'aspdotnetmvc', 'syntax', 'aspdotnet', 'postgresql', 'ajax', 'terminology', 'tsql', 'file', 'django', 'image', 'datetime', 'object', 'eclipse', 'dicionary', 'branch', 'gitbranch', 'twitterbootstrap', 'sorting', 'pointer', 'debugging', 'url', 'funcionalprogramming', 'date', 'scala', 'rfaq', 'datastructures', 'npm', 'googlechrome', 'dom', 'unittesting', 'funcion', 'authenticaion', 'maven', 'loop', 'inheritance', 'math', 'excepi

In [ ]:
df['new_tags_list'] = df['new_tags']

In [ ]:
df['new_tags_list'].apply(lambda x : x.split())

0                              [sqlserver, tsql]
1                                    [batchfile]
2                           [javascript, jquery]
3       [csharp, reflecion, optimizaion, lambda]
4                                         [java]
                          ...                   
8188                               [git, github]
8189                                    [docker]
8191                                    [window]
8192                                 [xcode, io]
8193                             [string, swift]
Name: new_tags_list, Length: 7747, dtype: object

In [ ]:
y_split = y.apply(lambda x: x.split())
vectorizer_target = CountVectorizer(tokenizer = None, binary='true')
y_bin = vectorizer_target.fit_transform(y)

In [ ]:
y_bin

<7747x198 sparse matrix of type '<class 'numpy.int64'>'
	with 14139 stored elements in Compressed Sparse Row format>

In [ ]:
pickle.dump(vectorizer_target, open('/content/drive/My Drive/Colab Notebooks/data_pickle/vectorizer_y.pickle', 'wb'))

In [ ]:
vectorizer_tfidf = TfidfVectorizer(analyzer="word",
                               min_df=MIN_DF,
                               stop_words=stop_words,
                               lowercase=True,
                               ngram_range=(1,2),
                               max_features=2000
                              )

In [ ]:
X_tttb_tfidf = vectorizer_tfidf.fit_transform(X_tttb)

In [ ]:
pickle.dump(vectorizer_tfidf, open('/content/drive/My Drive/Colab Notebooks/data_pickle/vectorizer_X.pickle', 'wb'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tttb_tfidf, y_bin, test_size = 0.2, random_state = 1)

In [ ]:
#mn = MultinomialNB()
svc = LinearSVC()
clf = OneVsRestClassifier(svc)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
# Save the model
pickle.dump(clf, open('/content/drive/My Drive/Colab Notebooks/data_pickle/model.pickle', 'wb'))

In [ ]:
type(X_test)

scipy.sparse.csr.csr_matrix

In [ ]:
#test
title = "Language Java programmation class ?"
body = "Language Java for progammation is very useful. Javadoc for object programmation and java developpers. Java."
#title = "SQl with mysql request error sql sql sql"
#body = "I have an error when I launch my request mysql. The sql request is : SELECT name, firstname from table where condition1 = blablabla group by. Do you have an udea ? thanks in advance. "

test_df = pd.DataFrame({'title_title_title_body': [title + title + body]},
                      index = ['1'])
test_df['title_title_title_body'] = test_df['title_title_title_body'].apply(lambda x: preprocessing_text(x))
#test_df['title_title_title_body'] = test_df['title_title_title_body'].apply(lambda x: keep_nouns_only(x))
test_df['title_title_title_body'] = test_df['title_title_title_body'].apply(lambda x: lemmatization(x))

In [ ]:
test_df['title_title_title_body']

1    language java programmaion class language java...
Name: title_title_title_body, dtype: object

In [ ]:
X_test_1 = vectorizer_tfidf.transform(test_df['title_title_title_body'])

In [ ]:
#X_test_1.toarray()

In [ ]:
y_pred = clf.predict(X_test_1)

In [ ]:
print(y_pred)

  (0, 100)	1


In [ ]:
result = vectorizer_target.inverse_transform(y_pred)

In [ ]:
print(result)

[array(['java'], dtype='<U20')]
